In [ ]:
import random as rad
#Khởi tạo mạng neuron
def init_network(n_input, n_hidden, n_output):
    network = list()
    #Trọng số của mỗi đầu vào n_input và bias đến lớp hidden
    hidden_layer = [{'weights':[rad.random() for i in range(n_input + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    #Trọng số của mỗi đầu vào n_hidden và bias đến lớp output
    output_layer = [{'weights':[rad.random() for j in range(n_hidden + 1)]} for j in range(n_output)]
    network.append(output_layer)
    return network
network = init_network(2,1,2)
print(network)

[[{'weights': [0.5852340056164083, 0.8484387075765376, 0.695023928397149]}], [{'weights': [0.8301924754376361, 0.1684450754870217]}, {'weights': [0.3850409681449646, 0.5987346759207254]}]]


In [ ]:
#Tính hàm kích hoạt cho mỗi đầu vào
def activation_input(weights, inputs):
    bias = weights[-1]
    activation = bias
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

In [ ]:
#Tính hàm sigmoid
import math as mat
def sigmoid(activation):
    return 1.0/(1.0 + mat.exp(-activation))

In [ ]:
#Tính lan truyền thẳng từ input đến output
#Forward Propagation
def forward_pro(network, data_row):
    inp = data_row
    for layer in network:
        temp = []
        for neuron in layer:
            activation = activation_input(neuron['weights'], inp)
            output_layer = sigmoid(activation)
            temp.append(output_layer)
            print(neuron)
        inp = temp
    return inp
print(forward_pro(network, [1,0]))

{'weights': [0.5852340056164083, 0.8484387075765376, 0.695023928397149]}
{'weights': [0.8301924754376361, 0.1684450754870217]}
{'weights': [0.3850409681449646, 0.5987346759207254]}
[0.6938255325029711, 0.7109550099539578]


In [ ]:
#Tính đạo hàm của hàm sigmoid ở đầu ra 
def sigmoid_derivative(output):
    return output*(1.0-output)

In [ ]:
#Tính hàm lỗi của backpropagation và lưu trong network
def backpropagation_error(network, expection):
    for j in reversed(range(len(network))):
        layer = network[j]
        errs = list()
        if j != len(network) - 1:
            for k in range(len(layer)):
                err = 0.0
                for neuron in network[j+1]:
                    err += (neuron['weights'][j]*neuron['error'])
                errs.append(err)
        else:
            for k in range(len(layer)):
                neuron = layer[k]
                errs.append(expection[k] - neuron['output'])
        for k in range(len(layer)):
            neuron = layer[k]
            neuron['error'] = errs[j] * sigmoid_derivative(neuron['output'])
network = [[{'output': 0.7105668883115941, 'weights': [0.9463936036227082, 0.6431101871813475, 0.4264771599829553]}],[{'output': 0.6213859615555266, 'weights': [0.9276732185160963, 0.6167567609526338]}, {'output': 0.6573693455986976, 'weights': [0.6731316543638922, 0.6552006657715309]}]]    
expection = [0, 1]
backpropagation_error(network,expection)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.9463936036227082, 0.6431101871813475, 0.4264771599829553], 'error': 0.02606269557370685}]
[{'output': 0.6213859615555266, 'weights': [0.9276732185160963, 0.6167567609526338], 'error': 0.08060915452180441}, {'output': 0.6573693455986976, 'weights': [0.6731316543638922, 0.6552006657715309], 'error': 0.0771723774346327}]
